In [1]:
from keybert import KeyBERT
import whisper_timestamped as whisper
from nltk import download
from nltk.corpus import stopwords

download("stopwords")
YT_LINK = "/content/vid test.weba"

Importing the dtw module. When using in academic works please cite:
  T. Giorgino. Computing and Visualizing Dynamic Time Warping Alignments in R: The dtw Package.
  J. Stat. Soft., doi:10.18637/jss.v031.i07.



[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Hardcoded text

In [2]:
with open("content/oleg.txt", "r", encoding="utf-8") as f:
    txt = f.read()

# NLP

In [3]:
from transformers import BertModel, BertTokenizerFast
from datasets import Dataset
import torch
import torch.nn.functional as F
import pandas as pd

In [4]:
model_checkpoint = 'bert-base-multilingual-cased'

tokenizer = BertTokenizerFast.from_pretrained(model_checkpoint)
model = BertModel.from_pretrained(model_checkpoint)
model = model.eval()

def get_embeddings(text):

  inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
  with torch.no_grad():
    outputs = model(**inputs)

  embeddings = outputs.pooler_output
  normalized_embeddings = F.normalize(embeddings, p=2)

  return normalized_embeddings

In [44]:
kw_model = KeyBERT()

tokens = txt.split(" ")
indexed_keywords = {}
df = pd.DataFrame(columns=["index", "keywords"])
for i in range(200, len(tokens), 200):
  frag = tokens[i-200: i]
  frag = " ".join(frag)

  doc_embeddings, word_embeddings = kw_model.extract_embeddings(frag)
  keywords = kw_model.extract_keywords(frag,
                                     doc_embeddings=doc_embeddings,
                                     word_embeddings=word_embeddings)
  indexed_keywords[i] = keywords

In [45]:
values_dict = {"index":[], "frags":[]}
for i, keywords in indexed_keywords.items():
  words = " ".join([k[0] for k in keywords])

  values_dict["index"].append(i)
  values_dict["frags"].append(words)

In [46]:
df = pd.DataFrame(values_dict)
df.head(5)

,index,frags
0,200,покидаю алгоритм годы комьюнити приятно
1,400,передовой школой сотрудничества подготовки сейчас
2,600,экспертизы экспертиза монокорпоративную формат...
3,800,формируется алгоритм компания продуктовыми про...
4,1000,инженерами доследовательской командной возможн...


In [47]:
words_dataset = Dataset.from_pandas(df)
words_dataset = words_dataset.map(
    lambda x: {'text_embeddings': get_embeddings(x["frags"])[0]}
)

Map:   0%|          | 0/19 [00:00<?, ? examples/s]

In [48]:
words_dataset.to_pandas().head()

,index,frags,text_embeddings
0,200,покидаю алгоритм годы комьюнити приятно,"[0.028846072, -0.022257075, 0.025537428, -0.01..."
1,400,передовой школой сотрудничества подготовки сейчас,"[0.035737332, -0.02563613, 0.025720645, -0.023..."
2,600,экспертизы экспертиза монокорпоративную формат...,"[0.032574058, -0.0206053, 0.020763565, -0.0272..."
3,800,формируется алгоритм компания продуктовыми про...,"[0.023455705, -0.005366963, 0.022595434, -0.01..."
4,1000,инженерами доследовательской командной возможн...,"[0.026208678, -0.0130467275, 0.014915336, -0.0..."


In [49]:
words_dataset = words_dataset.add_faiss_index(column='text_embeddings')

  0%|          | 0/1 [00:00<?, ?it/s]

In [55]:
test_frag_1 = words_dataset.to_pandas().head()['text_embeddings'][1]
words_dataset.to_pandas().head()

,index,frags,text_embeddings
0,200,покидаю алгоритм годы комьюнити приятно,"[0.028846072, -0.022257075, 0.025537428, -0.01..."
1,400,передовой школой сотрудничества подготовки сейчас,"[0.035737332, -0.02563613, 0.025720645, -0.023..."
2,600,экспертизы экспертиза монокорпоративную формат...,"[0.032574058, -0.0206053, 0.020763565, -0.0272..."
3,800,формируется алгоритм компания продуктовыми про...,"[0.023455705, -0.005366963, 0.022595434, -0.01..."
4,1000,инженерами доследовательской командной возможн...,"[0.026208678, -0.0130467275, 0.014915336, -0.0..."


In [63]:
indexed_keywords[200]

[('покидаю', 0.3988),
 ('алгоритм', 0.3596),
 ('годы', 0.3579),
 ('комьюнити', 0.3568),
 ('приятно', 0.3564)]

In [64]:
scores, samples = words_dataset.get_nearest_examples('text_embeddings',test_frag_1, k=15)
demo_df = pd.DataFrame({"index":samples['index']})
demo_df["score_to_1"] = scores
demo_df["keywords"] = demo_df["index"].apply(lambda x: [w[0] for w in indexed_keywords[x]])

In [65]:
demo_df

,index,score_to_5,keywords
0,400,0.000000,"[передовой, школой, сотрудничества, подготовки..."
1,3000,0.043668,"[профилю, роли, образовательный, хотели, биоте..."
2,3800,0.043825,"[взаимодействия, разной, конверсия, поисковике..."
3,2200,0.045507,"[нанимающими, какой, требования, компаниях, ко..."
4,600,0.046161,"[экспертизы, экспертиза, монокорпоративную, фо..."
5,1600,0.049893,"[взаимодействия, который, эксперты, исходим, с..."
6,1000,0.057495,"[инженерами, доследовательской, командной, воз..."
7,800,0.062978,"[формируется, алгоритм, компания, продуктовыми..."
8,1400,0.063792,"[новосибирск, инструментария, бизнес, бизнесом..."
9,1800,0.066138,"[помогают, программировали, приходит, возможно..."
